In [244]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.countplot(label)
plt.xlabel('label')

plt.title('RESOLUTION',fontsize=24)
plt.xlabel('Value',fontsize=14)
plt.ylabel('No. of resolution',fontsize=14)
plt.tick_params(axis='both',labelsize=14)
plt.show()

Exception ignored in: <function SeekableUnicodeStreamReader.__del__ at 0x7f4a6ff18268>
Traceback (most recent call last):
  File "/home/cr/anaconda3/lib/python3.7/site-packages/nltk/data.py", line 1161, in __del__
    if not self.closed:
  File "/home/cr/anaconda3/lib/python3.7/site-packages/nltk/data.py", line 1175, in closed
    return self.stream.closed
AttributeError: 'SeekableUnicodeStreamReader' object has no attribute 'stream'
/home/cr/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


NameError: name 'label' is not defined

In [1]:
# import Librarairs
import os,json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics import matthews_corrcoef
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_curve 

In [3]:
# Access Data
CWD = os.getcwd()  # get current directory path
BUG_REPORTS_DIR = os.path.join(CWD,"bugs")   # current directory+bugs
BUG_REPORTS_LIST = os.listdir(BUG_REPORTS_DIR)  # all the items in 'current_directory+bugs'

In [4]:

SENTI_REPORTS_DIR = os.path.join(CWD,"Sentiments_output")
SENTI_REPORTS_LIST = os.listdir(SENTI_REPORTS_DIR)

FileNotFoundError: [Errno 2] No such file or directory: '/home/cr/Desktop/ali/Implementation/Sentiments_output'

In [ ]:
# Load data set for bugs and sentiments


print(BUG_REPORTS_DIR)
print('-----------------')
print(BUG_REPORTS_LIST)
print('-----------------')
print(SENTI_REPORTS_DIR)
print('-----------------')
print(SENTI_REPORTS_LIST)         

In [5]:
# Extract data from files for sentiments
import csv
test_senti = []
train_senti=[]
import pandas as pd
for count, report in enumerate(SENTI_REPORTS_LIST):
    
    if count == 2:
        #add this project to test set
        print(f'Test Set: {report}')
        with open(os.path.join(SENTI_REPORTS_DIR,report), 'r') as report_file:
            temp_data = pd.read_csv(report_file)
            print(len(temp_data))
            #print(temp_data['neutral'])
            test_senti.extend(temp_data.iloc[0:, 1])
            
    else:
        #appent this project to train set
        print(f'Train Set: {report}')
        with open(os.path.join(SENTI_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = pd.read_csv(report_file)
            print(len(temp_data))
            #print(temp_data.iloc[0:, 1])
            train_senti.extend(temp_data.iloc[0:, 1])
            
            #print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")


NameError: name 'SENTI_REPORTS_LIST' is not defined

In [66]:
# Load data for bugs 
train_set = []
test_set = []

for count, report in enumerate(BUG_REPORTS_LIST):
    if count == 9:
        #add this project to test set
        print(f'Test Set: {report}')
        with open(os.path.join(BUG_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = json.load(report_file)
            print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")
            test_set.extend(temp_data["bugs"])
    else:
        #appent this project to train set
        print(f'Train Set: {report}')
        with open(os.path.join(BUG_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = json.load(report_file)
            print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")
            train_set.extend(temp_data["bugs"])   

Train Set: Core.json
File:Core.json <> Reports: 7228
Train Set: Toolkit.json
File:Toolkit.json <> Reports: 1899
Train Set: bugzilla.json
File:bugzilla.json <> Reports: 5324
Train Set: Calendar.json
File:Calendar.json <> Reports: 1785
Train Set: SeaMonkey.json
File:SeaMonkey.json <> Reports: 8716
Train Set: Camino Graveyard.json
File:Camino Graveyard.json <> Reports: 1290
Train Set: MailNews Core.json
File:MailNews Core.json <> Reports: 2796
Train Set: Firefox.json
File:Firefox.json <> Reports: 7368
Train Set: Thunderbird.json
File:Thunderbird.json <> Reports: 5129
Test Set: Core Graveyard.json
File:Core Graveyard.json <> Reports: 888


In [67]:
print(f'Train len: {len(train_set)}')
print(f'Test len: {len(test_set)}')
print(f'Total len: {len(train_set)+len(test_set)}')

Train len: 41535
Test len: 888
Total len: 42423


In [68]:
summary_train=[]
label_train=[]
summary_test=[]
label_test=[]
for i in train_set:
    summary_train.append(i['summary'])
    #summary_train=summary_train[0:10]
    label_train.append(i['resolution'])
print('train_summary',len(summary_train))
print('train_label',len(label_train))

    #label_train=label_train[0:10]
for j in test_set:
    summary_test.append(j['summary'])
    #summary_test=summary_test[0:10]
    label_test.append(j['resolution'])
   # label_test=label_test[0:10]
print('test_summary',len(summary_test))
print('test_label',len(label_test))
description_train=[]
description_test=[]

for k in train_set:
    description_train.append(k['description'])

for l in test_set:
    description_test.append(l['description'])

train_summary 41535
train_label 41535
test_summary 888
test_label 888


In [79]:
#Preprocess sentences
import re
def train_test_summary(summaries_text):
    
    lemmas=[]
    lemmatized_word=[]
    train_test_summaries=[]
    for count, summaries in enumerate (summaries_text):
        text = re.sub(r'[^a-zA-Z\']', ' ', summaries)
        #removes all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
        #substitute multiple spcaces with single space
        s_m_s = re.sub(r'\s+', ' ', document, flags=re.I)
        #removes prefix
        r_prefix = re.sub(r'^b\s+', '', s_m_s)
        #lemmas.append(r_prefix)
        r_pm = re.sub(r'[^\w\s]', '', r_prefix)
        step1 = r_pm.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
        step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
        step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
        step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
        step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
        "can not", "cannot")
        step6 = step5.replace(" ` ", " '")
        step7=step6.strip()
        step8=step7.lower()    
        lemmas=[]
        sentences = nltk.word_tokenize(step7)
        lemmatizer = WordNetLemmatizer()
        stem_words=[]
        for sentence_words in sentences:
            lower=sentence_words.lower()
            if lower not in stopwords.words('english'):
                lemma = lemmatizer.lemmatize(lower)
                stem_words.append(lemma)
            untokenized_sentence= ' '.join(stem_words)
        train_test_summaries.append(untokenized_sentence)
    return train_test_summaries

In [69]:
# Encocede Labels 
def train_test_labels(labels):
    train_test_label=[]
    for x in range(len(labels)):
        
        
        if label_train[x]=="FIXED":
            label_train[x]=1
        elif label_train[x]=="WONTFIX":
            label_train[x]=2
        elif label_train[x]=="INVALID":
            label_train[x]=3
        elif label_train[x]=="INCOMPLETE":
            label_train[x]=4
        elif label_train[x]=="WORKSFORME":
            label_train[x]=5
        elif label_train[x]=="EXPIRED":
            label_train[x]=6
        elif label_train[x]=="DUPLICATE":
            label_train[x]=7
        elif label_train[x]=="":
            label_train[x]=8
        elif label_train[x]=="MOVED":
            label_train[x]=9
        train_test_label.append(label_train[x])
        
    
    
    #print('train_lbel', len(train_label))
    return train_test_label

In [70]:

train_labels= train_test_labels(label_train)
test_labels=train_test_labels(label_test)

train_labels=np.array(train_labels)
test_labels=np.array(test_labels)


print(f"Training Labels Length:{len(train_labels)}")
print(f"Testing Labels Length:{len(test_labels)}")

print(f'Total len: {len(train_labels)+len(test_labels)}')

# Use 
#     train_labels
#     test_labels
test_lab, counts_elements =np.unique(test_labels, return_counts=True)
print(np.asarray((test_lab, counts_elements)))

Training Labels Length:41535
Testing Labels Length:888
Total len: 42423
[[  1   2   3   4   5   7   8]
 [282 159  72   2  49 258  66]]


In [243]:

train_summary=train_test_summary(summary_train)
test_summary=train_test_summary(summary_test)

print(f"Training summary Length:{len(train_summary)}")
print(f"Testing summary Length:{len(test_summary)}")



# Use 
#     train_summary
#     test_summary

KeyboardInterrupt: 

In [ ]:

d_train=train_test_summary(description_train)
d_test=train_test_summary(description_test)
print(f"Training summary Length:{len(d_train)}")
print(f"Testing summary Length:{len(d_test)}")
print(f'Total len: {len(d_train)+len(d_test)}')


In [257]:
vocab_size = 12000
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_l = 60 # max length of the sentence
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(summary_train)

word_index = tokenizer.word_index
print(word_index)

training_sequences = tokenizer.texts_to_sequences(summary_train)
training_padded1 = pad_sequences(training_sequences, maxlen=max_l, padding=padding_type, truncating=trunc_type)

print(training_padded1.shape)
testing_sequences = tokenizer.texts_to_sequences(summary_test)
testing_padded1= pad_sequences(testing_sequences, maxlen=max_l, padding=padding_type, truncating=trunc_type)
print(testing_padded1[1])
print(testing_padded1.shape)
print(type(testing_padded1))

{'<OOV>': 1, 'to': 2, 'in': 3, 'for': 4, 'the': 5, 'should': 6, 'a': 7, 'of': 8, 'and': 9, 'add': 10, 'on': 11, 'be': 12, 'rfe': 13, 'not': 14, 'new': 15, 'when': 16, 'mail': 17, 'from': 18, 'option': 19, 'tab': 20, 'window': 21, 'with': 22, 'menu': 23, 'is': 24, 'as': 25, 'message': 26, 'support': 27, 'search': 28, 'open': 29, 'page': 30, 'allow': 31, 'bar': 32, 'or': 33, 'folder': 34, 'text': 35, 'button': 36, 'use': 37, 'all': 38, 'tabs': 39, 'file': 40, 'an': 41, 'feature': 42, 'by': 43, 'bookmarks': 44, 'show': 45, 'address': 46, 'request': 47, 'firefox': 48, 'make': 49, 'mozilla': 50, 'messages': 51, 'have': 52, 'no': 53, 'need': 54, 'ability': 55, 'default': 56, 'view': 57, 'list': 58, 'download': 59, 'implement': 60, 'manager': 61, 'url': 62, 'bookmark': 63, 'dialog': 64, 'if': 65, 'toolbar': 66, 'user': 67, 'windows': 68, 'click': 69, 'it': 70, 'context': 71, 'like': 72, 'browser': 73, 'folders': 74, 'link': 75, 'display': 76, 'multiple': 77, 'that': 78, 'only': 79, 'filter': 

(37099, 60)
[1214 2720    1 1704    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
(5324, 60)
<class 'numpy.ndarray'>


In [258]:
vocab_size = 12000
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len= 60 # max length of the sentence
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(description_train)

word_index = tokenizer.word_index
#print(word_index)

training_sequences = tokenizer.texts_to_sequences(description_train)
training_padded2 = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

print(training_padded2.shape)
testing_sequences = tokenizer.texts_to_sequences(description_test)
testing_padded2 = pad_sequences(testing_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print(testing_padded2[1])
print(testing_padded2.shape)
print(type(testing_padded2))

(37099, 60)
[ 467   64    1    1    1  316  124   29 3283  215 1097 4843   69 1563
 1433  264 4257  585  711   53  148   14  171 3556    9  135  590   31
   11  428 1355  322    9 1582    2 1293   13    1  180  640  944   64
    1    1    1  316  124   29 3283  215 1097 4843   69 1779 3725  264
 4257  585  711    0]
(5324, 60)
<class 'numpy.ndarray'>


In [259]:
senti_summary_train= np.append(training_padded1,training_padded2,axis=1)
senti_summary_test= np.append(testing_padded1,testing_padded2,axis=1)
X1=senti_summary_train
Y1=senti_summary_test

print(X1.shape)
print(Y1.shape)

(37099, 120)
(5324, 120)


In [29]:

"""
#BI-LSTM
embedding_dim = 256

#model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()
"""

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 256)         2560000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 2,970,753
Trainable params: 2,970,753
Non-trainable params: 0
_________________________________________________________________


In [260]:
# CNN
from keras.models import Sequential
from keras import layers
from keras.layers import Flatten
from keras.optimizers import SGD
embedding_dim = 256

with tf.device('/GPU:0'):


    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=120))
    model.add(layers.Conv1D(128, 1, activation='tanh'))

    #model.add(layers.GlobalMaxPooling1D())
    #model.add(Flatten())
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(30, activation='tanh'))
    model.add(layers.Dense(1, activation='sigmoid'))



    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.summary()


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 120, 256)          3072000   
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 120, 128)          32896     
_________________________________________________________________
batch_normalization (BatchNo (None, 120, 128)          512       
_________________________________________________________________
dense_22 (Dense)             (None, 120, 30)           3870      
_________________________________________________________________
dense_23 (Dense)             (None, 120, 1)            31        
Total params: 3,109,309
Trainable params: 3,109,053
Non-trainable params: 256
_________________________________________________________________


In [62]:
#from keras.optimizers import Adam


#model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])
#model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
           #   optimizer=tf.keras.optimizers.Adam,
           # metrics=['accuracy'])
#Adagrad
#1e-08

In [261]:
with tf.device('/CPU:0'):
    num_epochs = 1
    batch_size = 32
    history = model.fit(X1, train_labels, epochs=num_epochs, batch_size=batch_size, verbose=1)

1160/1160 [==============================] - 93s 80ms/step - loss: 0.4437 - accuracy: 0.8321


In [162]:
# evaluate the model
_, train_accuracy = model.evaluate(X1, train_labels, verbose=0)
_, test_accuracy = model.evaluate(Y1,test_labels, verbose=0)
print(f"Train accuracy RNN:{train_accuracy*100}")
print(f"Test Accracy RNN:{test_accuracy*100}")

'''
Train accuracy RNN:80.70545792579651
Test Accracy RNN:68.21509003639221
'''

Train accuracy RNN:80.70545792579651
Test Accracy RNN:68.21509003639221


In [163]:
import numpy as np
yhat_prob = model.predict(Y1)
print(yhat_prob)

# get classes from probs
#yhat_class = np.argmax(yhat_prob, axis=-1) #for multilabel with softmax
yhat_class = (yhat_prob > 0.5).astype("int32") #for binary classes with sigmoid
#print(yhat_class[0:10])
# flaten pridicted and test lables into 1d-array
yhat = yhat_class[:,0]

print('done prepare yhat_prob & yhat_class')
#print(yhat_class[0:10])
#print(test_labels[0:10])


[[[0.32126805]
  [0.14489242]
  [0.18884811]
  ...
  [0.16407716]
  [0.20419231]
  [0.13273758]]

 [[0.26157904]
  [0.25496656]
  [0.16889343]
  ...
  [0.20357582]
  [0.20357582]
  [0.20357582]]

 [[0.25496656]
  [0.16281286]
  [0.25355664]
  ...
  [0.20357582]
  [0.20357582]
  [0.20357582]]

 ...

 [[0.17760518]
  [0.20057547]
  [0.15339565]
  ...
  [0.26780558]
  [0.19407913]
  [0.23001987]]

 [[0.25023967]
  [0.24056748]
  [0.08386877]
  ...
  [0.20357582]
  [0.20357582]
  [0.20357582]]

 [[0.19993556]
  [0.2601453 ]
  [0.20057547]
  ...
  [0.19716597]
  [0.20057547]
  [0.2173258 ]]]
done prepare yhat_prob & yhat_class


In [164]:
print(classification_report(test_labels,yhat))
print ('matthews_corrcoef:', matthews_corrcoef(test_labels,yhat))

              precision    recall  f1-score   support

           0       0.68      1.00      0.81       606
           1       0.00      0.00      0.00       282

    accuracy                           0.68       888
   macro avg       0.34      0.50      0.41       888
weighted avg       0.47      0.68      0.55       888

matthews_corrcoef: -0.02290479621335882


In [34]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
accuracy = metrics.accuracy_score(test_labels, yhat)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = metrics.precision_score(test_labels, yhat)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = metrics.recall_score(test_labels, yhat)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = metrics.f1_score(test_labels, yhat)
print('F1 score: %f' % f1)
 
# kappa
kappa = metrics.cohen_kappa_score(test_labels, yhat)
print('Cohens kappa: %f' % kappa)
# ROC AUC
auc = metrics.roc_auc_score(test_labels, yhat)
print('ROC AUC: %f' % auc)
# confusion matrix
matrix = metrics.confusion_matrix(test_labels, yhat)
print(matrix)

print(classification_report(y_true=test_labels, y_pred=yhat, digits=5))

Accuracy: 0.716754
Precision: 0.271739
Recall: 0.017053
F1 score: 0.032092
Cohens kappa: -0.000442
ROC AUC: 0.499843
[[3791   67]
 [1441   25]]
              precision    recall  f1-score   support

           0    0.72458   0.98263   0.83410      3858
           1    0.27174   0.01705   0.03209      1466

    accuracy                        0.71675      5324
   macro avg    0.49816   0.49984   0.43310      5324
weighted avg    0.59989   0.71675   0.61326      5324



In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(training_padded, label_train)
pred=clf.predict(testing_padded)

/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [198]:
#print(pred)
#print(test_label)
accuracy = metrics.accuracy_score(label_test, pred)
print('Accuracy: %f' % accuracy)
precision = metrics.precision_score(label_test, pred)
print('Precision: %f' % precision)
recall = metrics.recall_score(label_test, pred)
print('Recall: %f' % recall)
print(classification_report(y_true=label_test, y_pred=pred, digits=5))

Accuracy: 0.000000


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:182: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


ValueError: Mix of label input types (string and number)

In [149]:
import numpy as np
rng = np.random.RandomState(1)
from sklearn.naive_bayes import MultinomialNB
clfm = MultinomialNB()
clfm.fit(training_padded, label_train)
pred=clfm.predict(testing_padded)
accuracy = metrics.accuracy_score(label_test, pred)
print('Accuracy: %f' % accuracy)
precision = metrics.precision_score(label_test, pred)
print('Precision: %f' % precision)
recall = metrics.recall_score(label_test, pred)
print('Recall: %f' % recall)
print(classification_report(y_true=label_test, y_pred=pred, digits=5))

Accuracy: 0.485271
Precision: 0.255778
Recall: 0.478386
              precision    recall  f1-score   support

           0    0.71763   0.48780   0.58081       943
           1    0.25578   0.47839   0.33333       347

    accuracy                        0.48527      1290
   macro avg    0.48670   0.48310   0.45707      1290
weighted avg    0.59339   0.48527   0.51424      1290

